# Safety models with Elasticsearch Inference API & Hugging Face

This notebook demonstrates how to use Hugging Face completions along with the Elasticsearch Inference API. This notebook is based on the article [Safety models with Elasticsearch Inference API & Hugging Face](https://www.elastic.co/search-labs/blog/safety-models-inference-api-and-hugging-face).

In [17]:
%pip install requests elasticsearch -q


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


## Installing dependencies and importing packages

In [ ]:
import os
import json
import requests

from dotenv import load_dotenv
from elasticsearch import Elasticsearch, helpers

## Setting up environment variables

Configure API keys and URLs for Elasticsearch and Hugging Face, along with index name and inference endpoint identifiers.

In [ ]:
load_dotenv()

os.environ["ELASTICSEARCH_API_KEY"] = os.getenv("ELASTICSEARCH_API_KEY")
os.environ["ELASTICSEARCH_URL"] = os.getenv("ELASTICSEARCH_URL")
os.environ["HUGGING_FACE_API_KEY"] = os.getenv("HUGGING_FACE_API_KEY")
os.environ["HUGGING_FACE_INFERENCE_ENDPOINT_URL"] = os.getenv(
    "HUGGING_FACE_INFERENCE_ENDPOINT_URL"
)

INDEX_NAME = "community-blog-posts"
INFERENCE_ENDPOINT_ID = "hugging-face-gpt-oss-safeguard"

## Elasticsearch Python client

Initialize the Elasticsearch client using the configured URL and API key.

In [25]:
es_client = Elasticsearch(
    os.environ["ELASTICSEARCH_URL"],
    api_key=os.environ["ELASTICSEARCH_API_KEY"],
)

## Hugging Face completions inference endpoint setup

Create an Elasticsearch inference endpoint that connects to the Hugging Face model for generating responses based on blog posts and policies.

In [28]:
try:
    resp = es_client.inference.put(
        task_type="chat_completion",
        inference_id=INFERENCE_ENDPOINT_ID,
        body={
            "service": "hugging_face",
            "service_settings": {
                "api_key": os.environ["HUGGING_FACE_API_KEY"],
                "url": os.environ["HUGGING_FACE_INFERENCE_ENDPOINT_URL"],
            },
        },
    )

    print(
        "Chat completion inference endpoint created successfully:", resp["inference_id"]
    )
except Exception as e:
    print("Error creating chat completion inference endpoint:", {e})

Error creating chat completion inference endpoint: {BadRequestError('resource_already_exists_exception', meta=ApiResponseMeta(status=400, http_version='1.1', headers={'Content-Length': '283', 'Content-Type': 'application/vnd.elasticsearch+json;compatible-with=9', 'X-Cloud-Request-Id': 'VJsaAxn8S1i5XhsOr4bTew', 'X-Elastic-Product': 'Elasticsearch', 'X-Found-Handling-Cluster': 'de61f8bee3464185a51d8abb07400631.es', 'X-Found-Handling-Instance': 'es-es-index-7c64798c66-ft7nh', 'Date': 'Sun, 30 Nov 2025 16:43:34 GMT'}, duration=5.167906761169434, node=NodeConfig(scheme='https', host='articles-serverless-de61f8.es.us-central1.gcp.elastic.cloud', port=443, path_prefix='', headers={'user-agent': 'elasticsearch-py/9.2.0 (Python/3.11.13; elastic-transport/9.2.0)'}, connections_per_node=10, request_timeout=10.0, http_compress=False, verify_certs=True, ca_certs=None, client_cert=None, client_key=None, ssl_assert_hostname=None, ssl_assert_fingerprint=None, ssl_version=None, ssl_context=None, ssl_sh

## Ingestion flow

### Creating ingest pipeline for safety analysis

Create an ingest pipeline that uses the inference endpoint to evaluate each document during indexing, analyzing content against company policies and storing the result in `is_safe` and `safety_analysis` fields.

In [29]:
try:
    with open("policies.txt", "r", encoding="utf-8") as f:
        ruleset = f.read()
except FileNotFoundError:
    print("Warning: policies.txt not found. Using default ruleset.")
    ruleset = "Evaluate content for safety and compliance."

# Build the system prompt
system_prompt = f"""You are a content moderation assistant for a community blog platform. Evaluate submitted articles using the provided policy rules.

CRITICAL ANALYSIS REQUIREMENTS:
1. Detect BOTH explicit and implicit violations of policies
2. Analyze technical content for dangerous or malicious implications
3. Identify multi-layered issues (content violating multiple policies)
4. Consider context and intent, not just surface-level text

OUTPUT FORMAT:
Respond ONLY with valid JSON containing exactly two fields:
- "safe_content": boolean (true if safe, false if violates any rule)
- "moderation_analysis": string (detailed explanation citing specific policy violations or reasons for approval)

POLICY RULES:
{ruleset}"""

try:
    pipeline_body = {
        "description": "Evaluates blog post content for safety using inference endpoint",
        "processors": [
            {
                "script": {
                    "description": "Build prompt with article content",
                    "lang": "painless",
                    "source": "ctx.evaluation_prompt = params.system_prompt + ' Evaluate this article: Title: ' + ctx.title + ' Content: ' + ctx.content;",
                    "params": {"system_prompt": system_prompt},
                }
            },
            {
                "inference": {
                    "model_id": INFERENCE_ENDPOINT_ID,
                    "input_output": {
                        "input_field": "evaluation_prompt",
                        "output_field": "evaluation_result",
                    },
                }
            },
            {
                "json": {
                    "field": "evaluation_result",
                    "target_field": "evaluation_parsed",
                }
            },
            {
                "script": {
                    "description": "Extract fields from parsed JSON",
                    "lang": "painless",
                    "source": """
                        if (ctx.evaluation_parsed != null) {
                            ctx.safe_content = ctx.evaluation_parsed.safe_content;
                            ctx.moderation_analysis = ctx.evaluation_parsed.moderation_analysis;
                        } 
                    """,
                }
            },
            {
                "remove": {
                    "field": [
                        "evaluation_prompt",
                        "evaluation_result",
                        "evaluation_parsed",
                    ],
                    "ignore_missing": True,
                }
            },
        ],
    }

    PIPELINE_ID = "content-moderation-pipeline"

    es_client.ingest.put_pipeline(id=PIPELINE_ID, body=pipeline_body)
    print(f"Ingest pipeline '{PIPELINE_ID}' created successfully")
except Exception as e:
    print(f"Error creating ingest pipeline: {e}")

Ingest pipeline 'content-moderation-pipeline' created successfully


### Creating mappings

Define field types and properties including `semantic_text` with ELSER model for embeddings and `copy_to` properties for semantic search.

In [30]:
try:
    mapping = {
        "mappings": {
            "properties": {
                "id": {"type": "keyword"},
                "title": {
                    "type": "text",
                    "fields": {"keyword": {"type": "keyword"}},
                },
                "author": {"type": "keyword"},
                "category": {
                    "type": "keyword",
                },
                "content": {
                    "type": "text",
                },
                "date": {"type": "date"},
                "safe_content": {"type": "boolean"},
                "moderation_analysis": {"type": "text"},
            }
        }
    }

    es_client.indices.create(index=INDEX_NAME, body=mapping)
    print(f"Index {INDEX_NAME} created successfully")
except Exception as e:
    print(f"Error creating index: {e}")

Error creating index: BadRequestError(400, 'resource_already_exists_exception', 'index [community-blog-posts/PnuB1-9cSAaHD58cr7hgtg] already exists')


### Ingesting data to Elasticsearch

Use the bulk API to ingest the blog posts dataset into Elasticsearch from a JSON file. The ingest pipeline automatically evaluates each document for safety during indexing.

In [31]:
def build_data(json_file, index_name, pipeline_id=None):
    with open(json_file, "r") as f:
        data = json.load(f)

    for doc in data:
        action = {"_index": index_name, "_source": doc}
        if pipeline_id:
            action["pipeline"] = pipeline_id
        yield action


try:
    success, failed = helpers.bulk(
        es_client,
        build_data("dataset.json", INDEX_NAME, PIPELINE_ID),
    )
    print(f"{success} documents indexed successfully with safety analysis")


except Exception as e:
    print(f"Error: {str(e)}")

Error: Connection timed out


## results

### Semantic search

In [7]:
def execute_search(query, index_name, size=5):
    try:
        response = es_client.search(index=index_name, body=query, size=size)
        return response
    except Exception as e:
        print(f"Search error: {str(e)}")
        return None

Analyzing blogs with safety filters applied.

In [10]:
results = execute_search(
    query={
        "query": {
            "bool": {
                "filter": [{"term": {"safe_content": True}}],
            }
        }
    },
    index_name=INDEX_NAME,
    size=10,
)

for hit in results["hits"]["hits"]:
    source = hit["_source"]
    print(
        f"Title: {source['title']}\nContent: {source['content']}\nSafe Content: {source['safe_content']}\nModeration Analysis: {source['moderation_analysis']}\n"
        + "-" * 40
    )

Title: Security update: encrypted data pipelines
Content: We have enhanced our data pipelines with full encryption in transit and at rest. This aligns with our company’s commitment to protecting customer data.
Safe Content: True
Moderation Analysis: The article is a straightforward announcement of a public security enhancement. It describes features (encryption in transit and at rest) that are general, not revealing internal project codenames, internal pricing, or other confidential data. No personal or client information is disclosed, and the tone is respectful and factual. Therefore it complies with all policy criteria and is considered safe.
----------------------------------------
Title: Introducing the new Analytics Dashboard
Content: We are excited to announce the launch of our new analytics dashboard. This update introduces improved visualization tools and performance metrics for enterprise users. No confidential information is shared in this post.
Safe Content: True
Moderation 

/var/folders/5w/qq_73hhn37v6k0qn0gqscwsw0000gn/T/ipykernel_33141/2132694870.py:3: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es_client.search(index=index_name, body=query, size=size)


Analyzing blogs without safety filters.

In [11]:
results = execute_search(
    query={
        "query": {
            "bool": {
                "filter": [{"term": {"safe_content": False}}],
            }
        }
    },
    index_name=INDEX_NAME,
    size=10,
)

for hit in results["hits"]["hits"]:
    source = hit["_source"]
    print(
        f"Title: {source['title']}\nContent: {source['content']}\nSafe Content: {source['safe_content']}\nModeration Analysis: {source['moderation_analysis']}\n"
        + "-" * 40
    )

Title: Pricing details for enterprise accounts
Content: Here is a table with our internal pricing tiers and client discounts for Q4. These numbers are not public yet, but I think our readers will appreciate transparency.
Safe Content: False
Moderation Analysis: The article discloses internal pricing tiers and client discounts for Q4, which are not publicly released. This falls under the policy’s prohibition on sharing confidential financial information (Criteria 1 and 2). Therefore the content violates the company’s content policies and is not safe.
----------------------------------------
Title: Meet our competitors head-on!
Content: Unlike BrandX, our software is actually stable and bug-free. This article compares us directly to competitors and mocks their quality issues.
Safe Content: False
Moderation Analysis: The article contains a direct comparison to a competitor (BrandX) in a mocking tone, specifically stating that BrandX’s quality issues are inferior. This violates rule 3, whi

/var/folders/5w/qq_73hhn37v6k0qn0gqscwsw0000gn/T/ipykernel_33141/2132694870.py:3: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  response = es_client.search(index=index_name, body=query, size=size)


### Testing semantic search

In [ ]:
results = semantic_search(
    user_question="Search for updates related to data encryption.", size=3
)

print(f"Total hits: {results['total_hits']}\n")
for hit in results["hits"]:
    source = hit["_source"]
    print(f"Title: {source.get('title', 'N/A')}")
    print(f"Safe Content: {source.get('safe_content', 'Not evaluated')}")
    print(f"Moderation Analysis: {source.get('moderation_analysis', 'N/A')}")
    print(f"Content Preview: {source.get('content', '')[:200]}...")
    print("-" * 80)

### Testing search with safe content filter

In [ ]:
# Search only for safe content
safe_results = semantic_search(
    user_question="Find posts explaining debugging or authentication issues",
    size=3,
    filter_safe_only=True,
)

print(f"Safe content results: {safe_results['total_hits']}\n")
for hit in safe_results["hits"]:
    source = hit["_source"]
    print(f"✅ Title: {source.get('title', 'N/A')}")
    print(f"Analysis: {source.get('moderation_analysis', 'N/A')[:150]}...")
    print("-" * 80)

Safe content results: 0



## Generating completions function

Send messages to the Elasticsearch inference endpoint with streaming support, processing server-sent events to extract model responses in real-time.

In [24]:
def stream_chat_completion(messages: list, inference_id: str = INFERENCE_ENDPOINT_ID):
    try:

        response = requests.post(
            url=f"{os.environ['ELASTICSEARCH_URL']}/_inference/chat_completion/{inference_id}/_stream",
            json={
                "messages": messages,
            },
            headers={
                "Authorization": f"ApiKey {os.environ['ELASTICSEARCH_API_KEY']}",
                "Content-Type": "application/json",
            },
            stream=True,
        )
        response.raise_for_status()
        response.encoding = "utf-8"

        for line in response.iter_lines(decode_unicode=True):
            if line:
                print(line)
                line = line.strip()

                # Skip event lines like "event: message"
                if line.startswith("event:"):
                    continue

                # Process data lines
                if line.startswith("data: "):
                    data_content = line[6:]  # Remove "data: " prefix

                    if not data_content.strip() or data_content.strip() == "[DONE]":
                        continue

                    try:
                        chunk_data = json.loads(data_content)

                        # Extract the content from the response structure
                        if "choices" in chunk_data and len(chunk_data["choices"]) > 0:
                            choice = chunk_data["choices"][0]
                            if "delta" in choice and "content" in choice["delta"]:
                                content = choice["delta"]["content"]
                                if content:
                                    yield content

                    except json.JSONDecodeError as json_err:
                        print(f"\nJSON decode error: {json_err}")
                        print(f"Problematic data: {data_content}")
                        continue

    except requests.exceptions.RequestException as e:
        yield f"Error: {str(e)}"

In [34]:
response = moderation_chat(
    user_query="Find posts explaining debugging or authentication issues"
)

Moderation chat invoked with QUERY: Find posts explaining debugging or authentication issues and INFERENCE_ID: hugging-face-gpt-oss-safeguard
docs:  []
docs:  []
﻿event: message
data: {"id":"chatcmpl-5e22c0c5-b210-4044-8f46-03843d0a3123","choices":[{"delta":{"content":"The","role":"assistant"},"index":0}],"model":"gemma3:270m","object":"chat.completion.chunk"}
The﻿event: message
data: {"id":"chatcmpl-5e22c0c5-b210-4044-8f46-03843d0a3123","choices":[{"delta":{"content":" article"},"index":0}],"model":"gemma3:270m","object":"chat.completion.chunk"}
 article﻿event: message
data: {"id":"chatcmpl-5e22c0c5-b210-4044-8f46-03843d0a3123","choices":[{"delta":{"content":" violates"},"index":0}],"model":"gemma3:270m","object":"chat.completion.chunk"}
 violates﻿event: message
data: {"id":"chatcmpl-5e22c0c5-b210-4044-8f46-03843d0a3123","choices":[{"delta":{"content":" the"},"index":0}],"model":"gemma3:270m","object":"chat.completion.chunk"}
 the﻿event: message
data: {"id":"chatcmpl-5e22c0c5-b210-404

## Cleanup

Delete the index and inference endpoints to prevent consuming resources after completing the evaluation workflow.

In [14]:
# Cleanup - Delete Index
es_client.indices.delete(index=INDEX_NAME)

ObjectApiResponse({'acknowledged': True})

In [14]:
# Cleanup - Delete Inference Endpoints
es_client.ingest.delete_pipeline(id=PIPELINE_ID)
es_client.inference.delete(inference_id=INFERENCE_ENDPOINT_ID)


# es_client.inference.delete(inference_id=NEW_INFERENCE_ENDPOINT_ID)

ObjectApiResponse({'acknowledged': True, 'pipelines': [], 'indexes': []})